# IDR2 Delay Filtered Data Analysis

### Shane Bechtel & Tyler Cox

When imaging delay filtered IDR2 data, it was noted that the average visibility was much higher in images created using the full spectral range than images created using channels 100-800. Ideally when using the technique of delay filtering, all foregrounds should be removed. However, we are still seeing a visibility dependence on the foregrounds especially when using the full spectral range. This notebook is used to analyze the delay filtered IDR2 data to understand why we are seeing an increase in visibilites when using the entire spectral range. We start by importing the necessary libraries and selecting our plot settings.

In [5]:
% matplotlib notebook 

import numpy as np
import matplotlib.pyplot as plt
from pyuvdata import UVData
from matplotlib.colors import SymLogNorm
import os
import glob
from astropy.io import fits
from astropy.wcs import WCS
from astropy.time import Time

plt.rc('font', family='serif')
plt.rc('xtick', labelsize='small')
plt.rc('ytick', labelsize='small')

In [6]:
folders = np.sort(glob.glob('/data6/HERA/data/IDR2.1/LSTBIN/zen.grp1.of1.xx.LST.*.uvOCRSDL'))
folders_nf = np.sort(glob.glob('/data6/HERA/data/IDR2.1/LSTBIN/zen.grp1.of1.xx.LST.*.uvOCRSL'))

In [7]:
uvxy = UVData()

In [8]:
create_arr = True

for folder in folders_nf:
    print folder
    uvxy.read_miriad(folder)
    blin_len = np.sqrt(np.diag(np.dot(uvxy.uvw_array, uvxy.uvw_array.T)))
    blins_inds = np.argsort(blin_len)
    
    if create_arr:
        uvxy_delay_nf = np.fft.fft(uvxy.data_array,axis=2).squeeze()
        uvxy_delay_nf = uvxy_delay_nf[blins_inds].T
        create_arr = False
    else:
        wedge = np.fft.fft(uvxy.data_array,axis=2).squeeze()
        uvxy_delay_nf += wedge[blins_inds].T
        
    freq_width = np.diff(uvxy.freq_array[0,:])[0]
    delays_nf = np.fft.fftfreq(uvxy.Nfreqs,freq_width)*1e9
    

/data6/HERA/data/IDR2.1/LSTBIN/zen.grp1.of1.xx.LST.0.06682.uvOCRSL


Mean of empty slice.
invalid value encountered in double_scalars
antenna_diameters is not set. Using known values for HERA.


/data6/HERA/data/IDR2.1/LSTBIN/zen.grp1.of1.xx.LST.0.16078.uvOCRSL
/data6/HERA/data/IDR2.1/LSTBIN/zen.grp1.of1.xx.LST.0.25474.uvOCRSL
/data6/HERA/data/IDR2.1/LSTBIN/zen.grp1.of1.xx.LST.0.34870.uvOCRSL
/data6/HERA/data/IDR2.1/LSTBIN/zen.grp1.of1.xx.LST.0.44266.uvOCRSL
/data6/HERA/data/IDR2.1/LSTBIN/zen.grp1.of1.xx.LST.0.53661.uvOCRSL
/data6/HERA/data/IDR2.1/LSTBIN/zen.grp1.of1.xx.LST.0.63057.uvOCRSL
/data6/HERA/data/IDR2.1/LSTBIN/zen.grp1.of1.xx.LST.0.72453.uvOCRSL
/data6/HERA/data/IDR2.1/LSTBIN/zen.grp1.of1.xx.LST.0.81849.uvOCRSL
/data6/HERA/data/IDR2.1/LSTBIN/zen.grp1.of1.xx.LST.0.91245.uvOCRSL
/data6/HERA/data/IDR2.1/LSTBIN/zen.grp1.of1.xx.LST.1.00641.uvOCRSL
/data6/HERA/data/IDR2.1/LSTBIN/zen.grp1.of1.xx.LST.1.10036.uvOCRSL
/data6/HERA/data/IDR2.1/LSTBIN/zen.grp1.of1.xx.LST.1.19432.uvOCRSL
/data6/HERA/data/IDR2.1/LSTBIN/zen.grp1.of1.xx.LST.1.28828.uvOCRSL
/data6/HERA/data/IDR2.1/LSTBIN/zen.grp1.of1.xx.LST.1.38224.uvOCRSL
/data6/HERA/data/IDR2.1/LSTBIN/zen.grp1.of1.xx.LST.1.47620.uvO

KeyboardInterrupt: 

In [ ]:
create_arr = True

for folder in folders:
    print folder
    uvxy.read_miriad(folder)
    blin_len = np.sqrt(np.diag(np.dot(uvxy.uvw_array, uvxy.uvw_array.T)))
    blins_inds = np.argsort(blin_len)
    
    if create_arr:
        uvxy_delay_nf = np.fft.fft(uvxy.data_array,axis=2).squeeze()
        uvxy_delay_nf = uvxy_delay_nf[blins_inds].T
        create_arr = False
    else:
        wedge = np.fft.fft(uvxy.data_array,axis=2).squeeze()
        uvxy_delay_nf += wedge[blins_inds].T
        
    freq_width = np.diff(uvxy.freq_array[0,:])[0]
    delays = np.fft.fftfreq(uvxy.Nfreqs,freq_width)*1e9

In [ ]:
uvxy_delay_fft_nf = np.abs(np.fft.fftshift(uvxy_delay,axes=0))

In [ ]:
plt.figure(figsize=(8,6))

plt.imshow(uvxy_delay_fft_nf[:512]**2,
               aspect='auto',
               norm=SymLogNorm(vmin=1e3,vmax=1e7,linthresh=.1),
               extent=(blin_len.min(),blin_len.max(),delays.min(),delays.max()))
plt.xlabel('Baseline index ordered by length (~m?)')
plt.ylabel('Delay (ns)')
plt.colorbar()

In [ ]:
plt.figure(figsize=(8,6))

plt.imshow(uvxy_delay_fft[:512]**2,
               aspect='auto',
               norm=SymLogNorm(vmin=1e3,vmax=1e7,linthresh=.1),
               extent=(blin_len.min(),blin_len.max(),delays.min(),delays.max()))
plt.xlabel('Baseline index ordered by length (~m?)')
plt.ylabel('Delay (ns)')
plt.colorbar()

In [ ]:
plt.figure(figsize=(8,6))

plt.imshow(uvxy_delay_fft_nf[:512]**2-uvxy_delay_fft[:512]**2,
               aspect='auto',
               norm=SymLogNorm(vmin=1e3,vmax=1e7,linthresh=.1),
               extent=(blin_len.min(),blin_len.max(),delays.min(),delays.max()))
plt.xlabel('Baseline index ordered by length (~m?)')
plt.ylabel('Delay (ns)')
plt.colorbar()

In [35]:
(np.unique(uvxy.ant_1_array).shape[0]*np.unique(uvxy.ant_2_array).shape[0])

2704

In [30]:
uvxy.Nbls*60

82680